In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
!python --version

Python 3.7.4


In [3]:
import sys
import os
os.path.dirname(sys.executable)

'C:\\Users\\hp\\Anaconda3'

In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option("display.max_colwidth", 500)

In [27]:
movies_df = pd.read_csv("movies_metadata.csv", low_memory=False)

In [28]:
movies_df.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [29]:
movies_df.shape

(45466, 24)

In [30]:
movies_df.drop_duplicates(subset=['id'], inplace=True)

In [31]:
movies_df.shape

(45436, 24)

In [32]:
movies_df = movies_df.iloc[0:100]

In [12]:
movies_df['genres'] = movies_df['genres'].apply(lambda x: eval(x))

In [13]:
movies_df['production_companies'][4]

"[{'name': 'Sandollar Productions', 'id': 5842}, {'name': 'Touchstone Pictures', 'id': 9195}]"

In [14]:
movies_df.dropna(subset=['title'], inplace=True)

In [15]:
credits_df = pd.read_csv('credits.csv')
credits_df.drop_duplicates(subset=['id'], inplace=True)

In [16]:
credits_df= credits_df.iloc[0:100]

In [17]:
credits_df['cast'].isnull().sum()

0

In [18]:
movies_df.shape, credits_df.shape

((100, 24), (100, 3))

In [19]:
movies_df['id'] = movies_df['id'].astype(str)
credits_df['id'] = credits_df['id'].astype(str)

In [20]:
credits_df['cast'] = credits_df['cast'].apply(lambda x: eval(x))

# Prodcution Companies #

In [40]:
movies_df['production_companies'] = movies_df['production_companies'].apply(lambda x: eval(x))

XXXXXXXXXXXXXXXXX (ROUGH WORK)

In [296]:
credits_df = pd.read_csv("credits.csv")

In [191]:
credits_df.iloc[0]['cast'][0]

{'cast_id': 14,
 'character': 'Woody (voice)',
 'credit_id': '52fe4284c3a36847f8024f95',
 'gender': 2,
 'id': 31,
 'name': 'Tom Hanks',
 'order': 0,
 'profile_path': '/pQFoyx7rp09CJTAb932F2g8Nlho.jpg'}

In [234]:
# def _pri(head_type,head_id,head_properties,tail_type,tail_id,tail_properties,rel_type):
#     print(head_type, head_id, tail_type, tail_id)

In [236]:
# for i in x:
#     _pri(**i)
#     print("***")

In [41]:
for i in movies_df['budget']:
    try:
        float(i)
    except:
        print(i)

In [42]:
class MoviesPreCleaner:
    def driver(self, df):
        df = self.remove

In [43]:
movies_df[movies_df['budget']=='/ff9qCepilowshEtG2GYWwzt2bs4.jpg']

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count


In [44]:
sum(movies_df['budget']=='0')

48

In [45]:
ratings_df = pd.read_csv("ratings.csv")

In [46]:
 ratings_df

,userId,movieId,rating,timestamp
0,1,110,1.0,1425941529
1,1,147,4.5,1425942435
2,1,858,5.0,1425941523
3,1,1221,5.0,1425941546
4,1,1246,5.0,1425941556
...,...,...,...,...
26024284,270896,58559,5.0,1257031564
26024285,270896,60069,5.0,1257032032
26024286,270896,63082,4.5,1257031764
26024287,270896,64957,4.5,1257033990


In [239]:
# movies_df.head(1).T

XXXXXXXXXXXXXXXXX (END OF ROUGH WORK )

 ### Neo4J _ Knowledge Graph

In [47]:
from neo4j import GraphDatabase

In [48]:
# class HelloWorldExample:

#     def __init__(self, uri, user, password):
#         self.driver = GraphDatabase.driver(uri, auth=(user, password))

#     def close(self):
#         self.driver.close()
# #
#     def print_greeting(self, message):
#         with self.driver.session() as session:
#             greeting = session.write_transaction(self._create_and_return_greeting, message)
#             print(greeting)

#     @staticmethod
#     def _create_and_return_greeting(tx, message):
#         result = tx.run("CREATE (a:Greeting) "
#                         "SET a.message = $message "
#                         "RETURN a.message + ', from node ' + id(a)", message=message)
#         return result.single()[0]


# if __name__ == "__main__":
#     greeter = HelloWorldExample("bolt://localhost:7687", "neo4j", "admin")
#     greeter.print_greeting("hello, world")
#     greeter.close()

In [49]:
d = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "admin"), encrypted=False)

In [50]:
d

In [51]:
def name_property_check(func):
    def validate_func(**kwargs):
        print(kwargs)
        return func(*kwargs)
    return validate_func

In [52]:
class GraphUtils:
    def __init__(self):
        pass
    ## Utitlity to convert the dictionary object to into string ({'key1':'value1'} -> "{key1:'value'}")
    def _get_property_in_cypher_format(self, dic):
        format_string = "{"
        for k,v in dic.items():
            format_string += k + ':"%s", '%(v)
        format_string = format_string[:-2] # removing the last/trailing ", "
        format_string+="}"
        return format_string
    def insert_HTR(self, head_type,head_id,head_properties,tail_type,tail_id,tail_properties,rel_type,rel_properties=None):
        ## all the properties expcet rel_properties must have a 'id' field
        ## add a decorator here for validation
        head_properties = self._get_property_in_cypher_format(head_properties)
        tail_properties = self._get_property_in_cypher_format(tail_properties)
        ## for relation types make sure to check before that, that particular relation doesn't exist
        
        if rel_properties is None:
            query = """\
            MERGE(head: {head_type} {{ id:"{head_id}" }})
            SET head += {head_properties}
            MERGE(tail: {tail_type} {{ id:"{tail_id}" }}) 
            SET tail += {tail_properties}
            MERGE (head)-[rel: {rel_type}]->(tail) return head, tail, rel""".format(head_type=head_type,
                                                                                    head_id=head_id, 
                                                                                    head_properties=head_properties,
                                                                                    tail_type=tail_type, 
                                                                                    tail_id=tail_id,
                                                                                    tail_properties=tail_properties, 
                                                                                    rel_type=rel_type)
            ## initialize this d object with a singleton
            with d.session() as session:
                session.run(query)
                

In [53]:
from IPython.display import clear_output

In [54]:
class HRT_Extractor:
    def __init__(self):
        pass
    def _movie_genre_htr(self, mids, titles, genres):
        for m_id, genre_json_list, title in zip(mids, genres, titles):
            if len(genre_json_list)==0:
                continue
            for genre_json in genre_json_list:
                htr_dict = {
                       'head_type' : 'Movie',
                       'head_id' : str(m_id),
                       'head_properties' : {'id': str(m_id), 'title': str(title).replace('"',"'")},
                       'tail_type' : 'Genre',
                       'tail_id' : str(genre_json['id']),
                       'tail_properties' : genre_json,
                       'rel_type' : 'has_genre'
                   }
                yield htr_dict
                
    def movie_genre_population(self, movie_ids_list: list, movie_titles_list: list, genres: dict):
        idx = 0
        for _obj in self._movie_genre_htr(movie_ids_list, movie_titles_list, genres):
            GraphUtils().insert_HTR(**_obj)
            
            print(idx)
            if idx%100==0:
                clear_output()
            idx+=1
        clear_output()
        print(idx, "Completed")
        
    def _movie_actor_htr(self, mids, actors):
        for m_id, actor_json_list in zip(mids, actors):
            if len(actor_json_list)==0:
                continue
            for actor_json in actor_json_list:
                actor_json = {'id':actor_json['id'], 'name':actor_json['name'].replace('"',"'"), 'gender':str(actor_json['gender'])}
                if actor_json['name'] == """"Weird Al" Yankovic""" or actor_json['name'] == "'Weird Al' Yankovic":
                    print(actor_json)
                # whyyyyyy
                htr_dict = {
                       'head_type' : 'Movie',
                       'head_id' : str(m_id),
                       'head_properties' : {'id': str(m_id)},
                       'tail_type' : 'Actor',
                       'tail_id' : str(actor_json['id']),
                       'tail_properties' : actor_json,
                       'rel_type' : 'acted_by'
                   }
                yield htr_dict
                
    def movie_actor_population(self, movie_ids_list: list, actors: dict):
        idx = 0
        for _obj in self._movie_actor_htr(movie_ids_list,actors):
            GraphUtils().insert_HTR(**_obj)
            
            if idx%100==0:
                print(idx)
            if idx%1000==0:
                clear_output()
            idx+=1
        clear_output()
        print(idx, "Completed")
        
        
    def _movie_production_company_htr(self, mids, titles, prod_comps):
        for m_id, prod_comp_json_list, title in zip(mids, prod_comps, titles):
            if len(prod_comp_json_list)==0:
                continue
            for prod_comp_json in prod_comp_json_list:
                htr_dict = {
                       'head_type' : 'Movie',
                       'head_id' : str(m_id),
                       'head_properties' : {'id': str(m_id), 'title': str(title).replace('"',"'")},
                       'tail_type' : 'ProductionCompany',
                       'tail_id' : str(prod_comp_json['id']),
                       'tail_properties' : prod_comp_json,
                       'rel_type' : 'is_produced_by'
                   }
                yield htr_dict
                
    def movie_production_company_population(self, movie_ids_list: list, movie_titles_list: list, prod_comps: dict):
        idx = 0
        for _obj in self._movie_production_company_htr(movie_ids_list, movie_titles_list, prod_comps):
            GraphUtils().insert_HTR(**_obj)
            
            print(idx)
            if idx%100==0:
                clear_output()
            idx+=1
        clear_output()
        print(idx, "Completed")
        
        
        
    def _movie_year_htr(self, mids, titles, year):
        for m_id, year, title in zip(mids, year, titles):
            
            htr_dict = {
                       'head_type' : 'Movie',
                       'head_id' : str(m_id),
                       'head_properties' : {'id': str(m_id), 'title': str(title).replace('"',"'")},
                       'tail_type' : 'Year',
                       'tail_id' : str(year),
                       'tail_properties' : {'id': str(year), 'year': str(year)},
                       'rel_type' : 'released_in'
                   }
            yield  htr_dict
        
                
    def movie_year_population(self, movie_ids_list: list, movie_titles_list: list, year):
        idx = 0
        for _obj in self._movie_year_htr(movie_ids_list, movie_titles_list, year):
            GraphUtils().insert_HTR(**_obj)
            
            print(idx)
            if idx%100==0:
                clear_output()
            idx+=1
        clear_output()
        print(idx, "Completed")
        
        
                
    def _movie_director_htr(self, mids, directors):
        for m_id, director_json_list in zip(mids, directors):
            if len(director_json_list)==0:
                continue
            for director_json in director_json_list:
                director_json = {'id':director_json['id'], 'name':director_json['name'].replace('"',"'"), 'gender':str(director_json['gender'])}

                htr_dict = {
                       'head_type' : 'Movie',
                       'head_id' : str(m_id),
                       'head_properties' : {'id': str(m_id)},
                       'tail_type' : 'director',
                       'tail_id' : str(director_json['id']),
                       'tail_properties' : director_json,
                       'rel_type' : 'directed_by'
                   }
                yield htr_dict
                
    def movie_director_population(self, movie_ids_list: list, directors: dict):
        idx = 0
        for _obj in self._movie_director_htr(movie_ids_list,directors):
            GraphUtils().insert_HTR(**_obj)
            
            if idx%100==0:
                print(idx)
            if idx%1000==0:
                clear_output()
            idx+=1
        clear_output()
        print(idx, "Completed")

In [55]:
movies_df['genres']

0                                                                         [{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]
1                                                                        [{'id': 12, 'name': 'Adventure'}, {'id': 14, 'name': 'Fantasy'}, {'id': 10751, 'name': 'Family'}]
2                                                                                                         [{'id': 10749, 'name': 'Romance'}, {'id': 35, 'name': 'Comedy'}]
3                                                                            [{'id': 35, 'name': 'Comedy'}, {'id': 18, 'name': 'Drama'}, {'id': 10749, 'name': 'Romance'}]
4                                                                                                                                           [{'id': 35, 'name': 'Comedy'}]
                                                                                      ...                                                        

In [289]:
#HRT_Extractor().movie_genre_population([1,2,3], ['A', 'B', 'C'], movies_df['genres'].iloc[:4])

population Movie - Genre edges

In [59]:
HRT_Extractor().movie_genre_population(movies_df['id'], movies_df['title'], movies_df['genres'])

242 Completed


In [57]:
movies_df = pd.merge(movies_df, credits_df, how='inner', on='id')

In [67]:
#with d.session() as session:
 #   session.run("CREATE INDEX e FOR (a:Movie) ON (a.id)")
    
   

Creating Index on Movies

In [60]:
#with d.session() as session:
#    session.run("CREATE INDEX movie_id_index FOR (n:Movie) on (n.id)")

In [356]:
# def temp(mids, actors):
#         for m_id, actor_json_list in zip(mids, actors):
#             if len(actor_json_list)==0:
#                 continue
#             for actor_json in actor_json_list:
#                 actor_json = {'id':actor_json['id'], 'name':actor_json['name'].replace('"',"'"), 'gender':str(actor_json['gender'])}
#                 if actor_json['name'] == """"Weird Al" Yankovic""" or actor_json['name'] == "'Weird Al' Yankovic":
#                     print(actor_json)

In [357]:
#temp(movies_df['id'], movies_df['cast'])

population Movie - Actor edges

In [68]:
HRT_Extractor().movie_actor_population(movies_df['id'], movies_df['cast'])

1680 Completed


# Adding production companies #

In [69]:
HRT_Extractor().movie_production_company_population(movies_df['id'], movies_df['title'], movies_df['production_companies'])

250 Completed


In [ ]:
production_company

# Adding Time node #



In [70]:
movies_df['release_date']

0     1995-10-30
1     1995-12-15
2     1995-12-22
3     1995-12-22
4     1995-02-10
         ...    
95    1995-05-31
96    1994-12-06
97    1995-12-27
98    1996-02-16
99    1996-02-21
Name: release_date, Length: 100, dtype: object

In [71]:
from datetime import datetime
def Extract_year(x): 
    dt = datetime.strptime(x, '%Y-%m-%d')
    return dt.year

In [72]:
movies_df['release_year'] = movies_df['release_date'].apply(lambda x: Extract_year(x))

In [73]:
HRT_Extractor().movie_year_population(movies_df['id'], movies_df['title'], movies_df['release_year'])

100 Completed


In [90]:
movies_df['release_year'].nunique()

4

# Adding director and writer #

In [24]:
credits_df.head(3)

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)', 'credit_id': '52fe4284c3a36847f8024f95', 'gender': 2, 'id': 31, 'name': 'Tom Hanks', 'order': 0, 'profile_path': '/pQFoyx7rp09CJTAb932F2g8Nlho.jpg'}, {'cast_id': 15, 'character': 'Buzz Lightyear (voice)', 'credit_id': '52fe4284c3a36847f8024f99', 'gender': 2, 'id': 12898, 'name': 'Tim Allen', 'order': 1, 'profile_path': '/uX2xVf6pMmPepxnvFWyBtjexzgY.jpg'}, {'cast_id': 16, 'character': 'Mr. Potato Head (voice)', 'credit_id': '52fe4284c3a36847f8024...","[{'credit_id': '52fe4284c3a36847f8024f49', 'department': 'Directing', 'gender': 2, 'id': 7879, 'job': 'Director', 'name': 'John Lasseter', 'profile_path': '/7EdqiNbr4FRjIhKHyPPdFfEEEFG.jpg'}, {'credit_id': '52fe4284c3a36847f8024f4f', 'department': 'Writing', 'gender': 2, 'id': 12891, 'job': 'Screenplay', 'name': 'Joss Whedon', 'profile_path': '/dTiVsuaTVTeGmvkhcyJvKp2A5kr.jpg'}, {'credit_id': '52fe4284c3a36847f8024f55', 'department': 'Writing', 'gender': 2, 'id': 7, 'job': 'Screenplay', 'nam...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', 'credit_id': '52fe44bfc3a36847f80a7c73', 'gender': 2, 'id': 2157, 'name': 'Robin Williams', 'order': 0, 'profile_path': '/sojtJyIV3lkUeThD7A2oHNm8183.jpg'}, {'cast_id': 8, 'character': 'Samuel Alan Parrish / Van Pelt', 'credit_id': '52fe44bfc3a36847f80a7c99', 'gender': 2, 'id': 8537, 'name': 'Jonathan Hyde', 'order': 1, 'profile_path': '/7il5D76vx6QVRVlpVvBPEC40MBi.jpg'}, {'cast_id': 2, 'character': 'Judy Sheperd', 'credit_id': '52fe44bfc3a36847f8...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'department': 'Production', 'gender': 2, 'id': 511, 'job': 'Executive Producer', 'name': 'Larry J. Franco', 'profile_path': None}, {'credit_id': '52fe44bfc3a36847f80a7c89', 'department': 'Writing', 'gender': 2, 'id': 876, 'job': 'Screenplay', 'name': 'Jonathan Hensleigh', 'profile_path': '/l1c4UFD3g0HVWj5f0CxXAvMAGiT.jpg'}, {'credit_id': '52fe44bfc3a36847f80a7cdd', 'department': 'Sound', 'gender': 2, 'id': 1729, 'job': 'Original Music Composer', 'na...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'credit_id': '52fe466a9251416c75077a8d', 'gender': 2, 'id': 6837, 'name': 'Walter Matthau', 'order': 0, 'profile_path': '/xJVkvprOnzP5Zdh5y63y8HHniDZ.jpg'}, {'cast_id': 3, 'character': 'John Gustafson', 'credit_id': '52fe466a9251416c75077a91', 'gender': 2, 'id': 3151, 'name': 'Jack Lemmon', 'order': 1, 'profile_path': '/chZmNRYMtqkiDlatprGDH4BzGqG.jpg'}, {'cast_id': 4, 'character': 'Ariel Gustafson', 'credit_id': '52fe466a9251416c75077a95', 'gender...","[{'credit_id': '52fe466a9251416c75077a89', 'department': 'Directing', 'gender': 2, 'id': 26502, 'job': 'Director', 'name': 'Howard Deutch', 'profile_path': '/68Vae1HkU1NxQZ6KEmuxIpno7c9.jpg'}, {'credit_id': '52fe466b9251416c75077aa3', 'department': 'Writing', 'gender': 2, 'id': 16837, 'job': 'Characters', 'name': 'Mark Steven Johnson', 'profile_path': '/6trChNn3o2bi4i2ipgMEAytwmZp.jpg'}, {'credit_id': '52fe466b9251416c75077aa9', 'department': 'Writing', 'gender': 2, 'id': 16837, 'job': 'Writ...",15602


In [74]:
credits_df['crew'] = credits_df['crew'].apply(lambda x: eval(x))

In [75]:
len(credits_df['crew'][0])

106

In [120]:
for i in range (106):
    if (credits_df['crew'][0][i]['job']== 'Director'):
        print (credits_df['crew'][0][i])
    

{'credit_id': '52fe4284c3a36847f8024f49', 'department': 'Directing', 'gender': 2, 'id': 7879, 'job': 'Director', 'name': 'John Lasseter', 'profile_path': '/7EdqiNbr4FRjIhKHyPPdFfEEEFG.jpg'}


In [76]:
def MovieDirector(movie_crew):
    t=[]
    for i in range (len(movie_crew)):
        if (movie_crew[i]['job']== 'Director'):
            t.append(movie_crew[i])
    return (t)

In [77]:
MovieDirector(credits_df['crew'][99])

[{'credit_id': '52fe458b9251416c7505a35b',
  'department': 'Directing',
  'gender': 2,
  'id': 5655,
  'job': 'Director',
  'name': 'Wes Anderson',
  'profile_path': '/s03CeUeC5yAXyB1acqP0zGNo2SC.jpg'}]

In [78]:
movies_df['crew'] = movies_df['crew'].apply(lambda x : eval(x))

In [164]:
movies_df.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count', 'cast', 'crew'],
      dtype='object')

In [79]:
movies_df['Director'] = movies_df['crew'].apply(lambda x : MovieDirector(x))

In [80]:
HRT_Extractor().movie_director_population(movies_df['id'],  movies_df['Director'])

106 Completed


In [105]:
for i in range(100):
    print(movies_df[['genres','title' ]].iloc[i])


genres    [{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]
title                                                                                            Toy Story
Name: 0, dtype: object
genres    [{'id': 12, 'name': 'Adventure'}, {'id': 14, 'name': 'Fantasy'}, {'id': 10751, 'name': 'Family'}]
title                                                                                               Jumanji
Name: 1, dtype: object
genres    [{'id': 10749, 'name': 'Romance'}, {'id': 35, 'name': 'Comedy'}]
title                                                     Grumpier Old Men
Name: 2, dtype: object
genres    [{'id': 35, 'name': 'Comedy'}, {'id': 18, 'name': 'Drama'}, {'id': 10749, 'name': 'Romance'}]
title                                                                                 Waiting to Exhale
Name: 3, dtype: object
genres    [{'id': 35, 'name': 'Comedy'}]
title        Father of the Bride Part II
Name: 4, dtype: object
genres    [{'id

In [25]:
movies_df['genres']

0                                                                         [{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]
1                                                                        [{'id': 12, 'name': 'Adventure'}, {'id': 14, 'name': 'Fantasy'}, {'id': 10751, 'name': 'Family'}]
2                                                                                                         [{'id': 10749, 'name': 'Romance'}, {'id': 35, 'name': 'Comedy'}]
3                                                                            [{'id': 35, 'name': 'Comedy'}, {'id': 18, 'name': 'Drama'}, {'id': 10749, 'name': 'Romance'}]
4                                                                                                                                           [{'id': 35, 'name': 'Comedy'}]
                                                                                      ...                                                        

In [35]:
movies_df['genres'] = movies_df['genres'].apply(lambda x: eval(x))

TypeError: eval() arg 1 must be a string, bytes or code object

In [34]:
movies_df['genres']

0                                                                         [{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]
1                                                                        [{'id': 12, 'name': 'Adventure'}, {'id': 14, 'name': 'Fantasy'}, {'id': 10751, 'name': 'Family'}]
2                                                                                                         [{'id': 10749, 'name': 'Romance'}, {'id': 35, 'name': 'Comedy'}]
3                                                                            [{'id': 35, 'name': 'Comedy'}, {'id': 18, 'name': 'Drama'}, {'id': 10749, 'name': 'Romance'}]
4                                                                                                                                           [{'id': 35, 'name': 'Comedy'}]
                                                                                      ...                                                        